<a href="https://colab.research.google.com/github/FabioRovai/trove-test/blob/main/trove.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

At the time of writing there are 90 pages of data, 
we would like you to retrieve the GSID, Project Name, and SDG Certifications 
(SDG numbers only, not the corresponding SDG name) for all projects currently on the website.

In [ ]:
#Download selenium and chromium-chromedriver
!pip install selenium
#Update ubuntu to correctly run apt install
!apt-get update 
!apt install chromium-chromedriver
#Copy path
!cp /usr/lib/chromium-browser/chromedriver /usr/bin


Scraper takes about 8 minutes (pages loaded for less than 5 seconds tent to not load fully)

In [2]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException
import sys
import time
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
import re
import pandas as pd
#'''
##Setting selenium

#path driver (otherwise rise error in colab)
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
#choose chrome
chrome_options = webdriver.ChromeOptions()
#add args
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
#set scroll
javaScript = "window.scrollBy(0,1000);"
#set headless
options = Options()
options.headless = True
#Window sizing
options.add_argument("--window-size=1920,1200")
#Activate JS !! - It will not work if this is not enabled
options.add_argument("javascript.enabled")
#set webdriver
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

##SCRAPING

#Set Counter to scroll each of the 90 pages 
counter=1
while counter <= 90:
  filename = 'https://registry.goldstandard.org/projects/?q=&page='
  path = filename + str(counter) 
  driver.get(path)
  driver.execute_script(javaScript)
  driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")  
  time.sleep(5) #add delay, quite slower but works (5 for debug)  
  counter += 1
  file = open("sample.txt", "a")
  str_dictionary = repr(driver.page_source)
  file.write("new_page = " + str_dictionary + "\n")

 
driver.quit()#'''

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: DeprecationWarning: use options instead of chrome_options


In [3]:
##GSID

#read file back
text_file = open("/content/sample.txt", "r") 
lines = text_file.readlines()
#split by space
words = str(lines).split()
#Tokenizer set
start = 'colspan="1">GS'
#make filter
filter_object = filter(lambda a: start in a, words)
filter_object = list(filter_object)
#apply strip to list
baz=[i.strip('class="jss451">') for i in filter_object]
foo=[i.strip('colspan="1">') for i in baz]
bar=[i.strip('</td><td') for i in foo]
GS_ID=pd.DataFrame(bar)
GS_ID.to_csv('GSID.csv')



In [4]:
##Project Name

text_file = open("/content/sample.txt", "r")
lines = text_file.readlines()
#split by word 'class'
start2='class' 
words2 = str(lines).split(start2)
#match "jss451"> identifier
matches = [match for match in words2 if '"jss451">' in match] 
#strip lists
names=[i.strip('="jss451">') for i in matches]
names_clean=[i.strip(' </div><div') for i in names]
names_clean
ProjectName=pd.DataFrame(names_clean)
ProjectName.to_csv('ProjectName.csv')

In [5]:
##SDG Certifications

text_file = open("/content/sample.txt", "r")
lines = text_file.readlines()
#find word class="jss450"
lines= str(lines)[str(lines).find('class="jss450"'):]
#split by word class="jss450"
words3 = str(lines).split('class="jss450"')
#match word img
matches2 = [match for match in words3 if 'img' in match]
#create list of lists
qux=[i.split('jss456"><img') for i in matches2]
#set counter
digit=0
while digit <= len(qux)-1:
  quux=[i.split('-', 1)[0]for i in qux[digit]]
  del quux[0]
  quux=str(quux)
  numeric_string = re.sub("[^0-9]", " ", quux)
  numeric_string = re.sub(" +", " ", numeric_string) 
  digit += 1
  file = open("numbers.txt", "a")
  file.write(numeric_string + "\n")

#small debug for the last line of the DS
quux=[i.split('-', 1)[0]for i in qux[90]]
del quux[0]
quux=str(quux)
numeric_string = re.sub("[^0-9]", " ", quux)
numeric_string = re.sub(" +", " ", numeric_string) 

file = open("numbers.txt", "a")
file.write(numeric_string + "\n")
#len(numeric_string)

11

In [6]:
##Cleaning SDG Certifications

text_file = open("/content/numbers.txt", "r")
SDG = text_file.readlines()
SDG=pd.DataFrame(SDG)
SDG.to_csv('numbers.csv')
SDG=pd.read_csv('numbers.csv')
#rename
SDG.rename(columns={ SDG.columns[1]: "SDG_Certifications" }, inplace = True)

SDG.head()

,Unnamed: 0,SDG_Certifications
0,0,8 12 13 \n
1,1,3 5 6 13 \n
2,2,3 5 6 13 \n
3,3,3 5 6 13 \n
4,4,7 8 9 10 11 12 13 \n


In [7]:
##create a csv
GS_ID.rename(columns={ GS_ID.columns[0]: "GSID" }, inplace = True)
ProjectName.rename(columns={ ProjectName.columns[0]: "Project_Name" }, inplace = True)
result = pd.concat([GS_ID, ProjectName,SDG], axis=1)
result=result.drop(['Unnamed: 0'], axis=1)
result['SDG_Certifications'] = result['SDG_Certifications'].str.strip('\n')
result

,GSID,Project_Name,SDG_Certifications
0,GS11255,Muyuan Nanyang Swine Farm Animal Waste Managem...,8 12 13
1,GS11250,GS1247 VPA 315 GivePower Haiti Solar Water Farms,3 5 6 13
2,GS11249,GS1247 VPA 313 GivePower Haiti Solar Water Farms,3 5 6 13
3,GS11248,GS1247 VPA 314 GivePower Haiti Solar Water Farms,3 5 6 13
4,GS11239,Shangrao Swine Farm Animal Manure Management S...,7 8 9 10 11 12 13
...,...,...,...
2230,GS1006,Highveld Air Quality - Highveld Central Project,7 8 13
2231,GS1005,GS1005 Qori Q\\\'oncha - Improved cookstoves d...,3 13 15
2232,GS1003,Degirmen Weir and Hydroelectric Power Plant,7 9 13
2233,GS1001,InfraVest Taiwan Wind Farms Bundled Project 20...,7 8 13


In [8]:
##Dump csv
result.to_csv('result.csv')